# Describe the click distribution of user on Map using folium

Install dependecies

In [1]:
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install osmnet
!pip install matplotlib==3.4.3
!pip install pandana
!pip install folium

Import libraries

In [2]:
import geopandas as gpd
import folium
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from folium import plugins
from folium.plugins import HeatMap

Cloning data

In [3]:
! git clone https://github.com/CityScope/CSL_HCMC.git

fatal: destination path 'CSL_HCMC' already exists and is not an empty directory.


Read districts data

In [4]:
data_districts = gpd.read_file('/content/CSL_HCMC/Data/GIS/Population/population_HCMC/population_shapefile/Population_District_Level.shp')
data_districts=data_districts.to_crs(epsg=4326)
data_districts.head()

,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,geometry
0,District 1,760,District,180225.0,187435.0,187419.0,190943.0,197421.0,200297.0,193632.0,208475.0,211220.0,142625.0,23360.40,24294.90,24292.90,24749.60,25589.30,25962.10,25098.20,27022.1,27377.90,18486.8,15274.835115,7.714899e+06,"POLYGON ((106.69160 10.79553, 106.69181 10.795..."
1,District 3,770,District,190553.0,188945.0,190631.0,191580.0,193206.0,193694.0,196333.0,198104.0,199095.0,190375.0,38688.60,38362.10,38704.40,38897.10,39227.20,39326.30,39862.10,40221.7,40422.90,38652.5,12182.058264,4.925215e+06,"POLYGON ((106.68641 10.79229, 106.68667 10.792..."
2,District 4,773,District,180980.0,183261.0,184711.0,185649.0,185808.0,187157.0,186727.0,186995.0,183894.0,175329.0,43311.10,43857.00,44204.00,44428.40,44466.50,44789.30,44686.40,44750.6,44008.40,41958.7,9483.716891,4.178668e+06,"POLYGON ((106.71584 10.75245, 106.71574 10.752..."
3,District 6,775,District,249329.0,253474.0,254212.0,257865.0,266121.0,257183.0,258945.0,260742.0,262757.0,233561.0,34910.50,35490.80,35594.20,36105.70,37261.60,36010.20,36256.90,36508.5,36790.60,32702.7,13077.365262,7.140945e+06,"POLYGON ((106.64179 10.73826, 106.63760 10.735..."
4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,317488.0,327223.0,360155.0,6929.39,7796.06,7616.07,7563.59,7963.85,8418.12,8798.84,9006.2,9282.35,10216.5,33260.915360,3.525220e+07,"POLYGON ((106.75137 10.76062, 106.75001 10.758..."


Population raising from 2017 to 2019

In [5]:
pop_2019=data_districts['Pop_2019']
pop_2017=data_districts['Pop_2017']

pop_raising=pop_2019/pop_2017

Top districts with highest population raise

In [6]:
top10_raise=pop_raising.sort_values(ascending=False)[:10]
top10_idx=top10_raise.index
top10_geometry=data_districts['geometry'][top10_idx]
top10_geometry

16    POLYGON ((106.84015 10.89893, 106.84048 10.897...
23    POLYGON ((106.58361 10.92869, 106.58391 10.928...
18    POLYGON ((106.72315 10.72162, 106.72428 10.721...
22    POLYGON ((106.69433 10.89842, 106.69446 10.897...
17    POLYGON ((106.74241 10.80855, 106.74442 10.808...
13    POLYGON ((106.43739 11.10990, 106.43727 11.109...
4     POLYGON ((106.75137 10.76062, 106.75001 10.758...
15    POLYGON ((106.72733 10.89026, 106.72753 10.890...
14    POLYGON ((106.64275 10.72768, 106.64323 10.727...
12    POLYGON ((106.60037 10.82610, 106.60044 10.826...
Name: geometry, dtype: geometry

Read click data and convert to **32648 format**

In [7]:
top10_district_id=data_districts['Dist_ID'][top10_idx]
top10_district_name = data_districts['Dist_Name'][top10_idx]
top10_district_id

16    763
23    784
18    786
22    761
17    769
13    783
4     778
15    762
14    785
12    777
Name: Dist_ID, dtype: object

In [8]:
click_data = gpd.read_file('/content/response3.json')
click_data=click_data.to_crs(epsg=4326)
click_data

,dbh,geometry
0,1,POINT (106.71136 10.85113)
1,1,POINT (106.71695 10.74449)
2,1,POINT (106.66315 10.83289)
3,1,POINT (106.66267 10.83339)
4,1,POINT (106.65977 10.83729)
...,...,...
83670,1,POINT (106.65664 10.84339)
83671,1,POINT (106.75180 10.85276)
83672,1,POINT (106.73189 10.73154)
83673,1,POINT (106.67640 10.82388)


Join data between click and district

In [9]:
join=gpd.sjoin(click_data,data_districts,how='left',op='within')
join

,dbh,geometry,index_right,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area
0,1,POINT (106.71136 10.85113),15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.20,10487.80,10630.20,10842.20,11065.00,11245.9,11362.30,12410.9,52828.501430,4.775581e+07
1,1,POINT (106.71695 10.74449),4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,317488.0,327223.0,360155.0,6929.39,7796.06,7616.07,7563.59,7963.85,8418.12,8798.84,9006.2,9282.35,10216.5,33260.915360,3.525220e+07
2,1,POINT (106.66315 10.83289),10,Go Vap,764,District,522690.0,548145.0,566216.0,588606.0,604143.0,620078.0,634146.0,646060.0,663313.0,676899.0,26510.80,27801.80,28718.40,29854.00,30642.00,31450.30,32163.80,32768.1,33643.10,34332.2,27550.300914,1.971611e+07
3,1,POINT (106.66267 10.83339),10,Go Vap,764,District,522690.0,548145.0,566216.0,588606.0,604143.0,620078.0,634146.0,646060.0,663313.0,676899.0,26510.80,27801.80,28718.40,29854.00,30642.00,31450.30,32163.80,32768.1,33643.10,34332.2,27550.300914,1.971611e+07
4,1,POINT (106.65977 10.83729),10,Go Vap,764,District,522690.0,548145.0,566216.0,588606.0,604143.0,620078.0,634146.0,646060.0,663313.0,676899.0,26510.80,27801.80,28718.40,29854.00,30642.00,31450.30,32163.80,32768.1,33643.10,34332.2,27550.300914,1.971611e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83670,1,POINT (106.65664 10.84339),10,Go Vap,764,District,522690.0,548145.0,566216.0,588606.0,604143.0,620078.0,634146.0,646060.0,663313.0,676899.0,26510.80,27801.80,28718.40,29854.00,30642.00,31450.30,32163.80,32768.1,33643.10,34332.2,27550.300914,1.971611e+07
83671,1,POINT (106.75180 10.85276),15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.20,10487.80,10630.20,10842.20,11065.00,11245.9,11362.30,12410.9,52828.501430,4.775581e+07
83672,1,POINT (106.73189 10.73154),4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,317488.0,327223.0,360155.0,6929.39,7796.06,7616.07,7563.59,7963.85,8418.12,8798.84,9006.2,9282.35,10216.5,33260.915360,3.525220e+07
83673,1,POINT (106.67640 10.82388),10,Go Vap,764,District,522690.0,548145.0,566216.0,588606.0,604143.0,620078.0,634146.0,646060.0,663313.0,676899.0,26510.80,27801.80,28718.40,29854.00,30642.00,31450.30,32163.80,32768.1,33643.10,34332.2,27550.300914,1.971611e+07


Grouping result by districts

In [10]:
top10_district_id.columns=['Dist_ID']
top10_district_id

16    763
23    784
18    786
22    761
17    769
13    783
4     778
15    762
14    785
12    777
Name: Dist_ID, dtype: object

In [11]:
join=join.merge(top10_district_id)
join

,dbh,geometry,index_right,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area
0,1,POINT (106.71136 10.85113),15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.2,10487.80,10630.20,10842.20,11065.00,11245.9,11362.30,12410.90,52828.501430,4.775581e+07
1,1,POINT (106.75131 10.85530),15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.2,10487.80,10630.20,10842.20,11065.00,11245.9,11362.30,12410.90,52828.501430,4.775581e+07
2,1,POINT (106.75553 10.84865),15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.2,10487.80,10630.20,10842.20,11065.00,11245.9,11362.30,12410.90,52828.501430,4.775581e+07
3,1,POINT (106.75589 10.84450),15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.2,10487.80,10630.20,10842.20,11065.00,11245.9,11362.30,12410.90,52828.501430,4.775581e+07
4,1,POINT (106.75193 10.85237),15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.2,10487.80,10630.20,10842.20,11065.00,11245.9,11362.30,12410.90,52828.501430,4.775581e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45193,1,POINT (106.68526 10.73004),14,Binh Chanh,785,District,420109.0,447291.0,469517.0,491900.0,514242.0,551545.0,591451.0,623022.0,652900.0,705508.0,1660.96,1768.42,1856.3,1944.79,2033.12,2180.61,2338.38,2463.2,2581.33,2789.32,129933.645113,2.529152e+08
45194,1,POINT (106.67969 10.73197),14,Binh Chanh,785,District,420109.0,447291.0,469517.0,491900.0,514242.0,551545.0,591451.0,623022.0,652900.0,705508.0,1660.96,1768.42,1856.3,1944.79,2033.12,2180.61,2338.38,2463.2,2581.33,2789.32,129933.645113,2.529152e+08
45195,1,POINT (106.67874 10.73188),14,Binh Chanh,785,District,420109.0,447291.0,469517.0,491900.0,514242.0,551545.0,591451.0,623022.0,652900.0,705508.0,1660.96,1768.42,1856.3,1944.79,2033.12,2180.61,2338.38,2463.2,2581.33,2789.32,129933.645113,2.529152e+08
45196,1,POINT (106.63395 10.66310),14,Binh Chanh,785,District,420109.0,447291.0,469517.0,491900.0,514242.0,551545.0,591451.0,623022.0,652900.0,705508.0,1660.96,1768.42,1856.3,1944.79,2033.12,2180.61,2338.38,2463.2,2581.33,2789.32,129933.645113,2.529152e+08


Draw HCMC map

In [12]:
map = folium.Map(location=[10.72, 106.62], zoom_start=11, tiles='CartoDB positron')
map

**Draw heat function**

In [13]:
def add_heat(df):
    kmeans = KMeans(n_clusters=20)
    a=pd.Series(df['geometry'].apply(lambda p: p.x))
    b=pd.Series(df['geometry'].apply(lambda p: p.y))
    X=np.column_stack((a,b))
    y_kmeans = kmeans.fit_predict(X)
    df['cluster'] = y_kmeans

    max_cluster = df.groupby(['cluster']).size().idxmax()

    max_click = df[df['cluster'] == max_cluster]
    a=pd.Series(max_click['geometry'].apply(lambda p: p.x))
    b=pd.Series(max_click['geometry'].apply(lambda p: p.y))
    max_click['lat'] = b
    max_click['lon'] = a

    heat_data = [[row['lat'],row['lon']] for index, row in max_click.iterrows()]
    HeatMap(heat_data).add_to(map)

Draw heatmap

In [14]:
for district_id in top10_district_id:
    data = join[join['Dist_ID'] == district_id]
    add_heat(data)

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

Show map

In [15]:
map

Export to html file

In [16]:
map.save('result.html')

Download the fhantomjs executable

In [17]:
!pip install selenium

In [18]:
! chmod +x /content/phantomjs

In [19]:
import selenium.webdriver

driver = selenium.webdriver.PhantomJS('/content/phantomjs')
driver.set_window_size(3000, 2000)  # choose a resolution
driver.get('result.html')
# You may need to add time.sleep(seconds) here
driver.save_screenshot('image.png')

/usr/local/lib/python3.7/dist-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


True